# 05. 선형대수와 Numpy 프로그래밍 (3)
> 머신러닝에 꼭 필요한 선형대수학 개념들을 배우고 이를 Numpy로 구현하는 방법을 배워봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 1]
- permalink: /linear_algebra_with_numpy_3
- exec: colab

이제 여러분은 선형대수의 프레임워크로 생각할 수 있게 되었습니다. 이제 행렬보다 상위 차원 존재인 텐서에 대해 배우고, 데이터를 자르고 붙이고 돌리는 방법들에 대해 배워봅시다. 이러한 함수들은 실제로 꽤나 유용하며, 이런 함수들을 잘 모르면 산술 프로그래밍 자체가 불가능하니 잘 봐주셔야합니다.

### 1. Tensor

행렬은 기본적으로 벡터가 여러개 쌓여서 만들어진 구조입니다. (기저벡터들이 변환되고 난 뒤의 위치를 가로로 쌓아놓은 구조) 따라서 (n) 혹은 (n, 1)의 Shape를 갖는 벡터들이 m개 쌓이면 Shape이 (n, m)인 행렬이 되는 방식이였습니다.

![img](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day1/70.png?raw=true)

그러나 이러한 행렬이 쌓여서 3차원 큐브와 같은 자료를 만들면 어떻게 될까요? 또 그 3차원 큐브가 쌓여서 4차원 데이터를 만들어내면 어떻게 될까요? 이 때 데이터 자체가 몇개의 방향(차원)으로 존재하는지를 우리는 "랭크(Rank)"라고 부릅니다. <br><br>

![img](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day1/69.png?raw=true)

스칼라는 랭크가 0, 벡터는 랭크가 1, 행렬은 랭크가 2입니다. 그리고 랭크가 3이상인 모든 데이터들을 "텐서"라고 부르며, 우리는 앞으로 이 텐서를 다룰 것입니다.
<br><br>

### 2. Tensor의 Shape 규칙
Tensor의 경우 Rank가 높기 때문에 Shape을 맞추기 까다롭습니다. 아래와 같은 규칙이 존재합니다.
<br>

![](https://2.bp.blogspot.com/-iuS-Uayk2FA/T_VUQ3nvaiI/AAAAAAAAAhs/ARvCwQFufsc/s1600/matrix_multi.png)
<br><br>

#### 2.1. 3차원 텐서 Shape 규칙
(x, i, j) @ (x, j, k)을 연산할 때 x는 같거나 양쪽 중 한쪽이 1이여야 하며, 맨 끝 두개의 Shape은 (i, j) @ (j, k)에서 j는 동일해야하고, 결과 Shape은 (x, i, k)가 된다.
<br>

- (4, 1, 2) @ (4, 2, 5) = (4, 1, 5)
- (4, 1, 2) @ (3, 2, 5) = 가장 앞자리가 달라서 에러
- (1, 1, 2) @ (3, 2, 5) = (3, 1, 5) 
<br><br>

#### 2.2. 4차원 텐서 Shape 규칙
(x, y, i, j) @ (x, y, j, k)을 연산할 때 x, y는 같거나 양쪽 중 한쪽이 1이여야 하며, 맨 끝 두개의 Shape은 (i, j) @ (j, k)에서 j는 동일해야하고, 결과 Shape은 (x, y, i, k)가 된다.
<br>

- (4, 3, 1, 2) @ (4, 3, 2, 5) = (4, 3, 1, 5)
- (4, 5, 1, 2) @ (3, 5, 2, 5) = 가장 앞자리가 달라서 에러
- (4, 5, 1, 2) @ (4, 3, 2, 5) = 두번째 자리가 달라서 에러
- (4, 5, 1, 2) @ (4, 1, 2, 5) = (4, 5, 1, 5)
- (4, 5, 1, 2) @ (1, 5, 2, 5) = (4, 5, 1, 5)
<br><br>

#### 2.3. n차원 텐서 Shape 규칙
위를 토대로 아래와 같은 일반화가 가능합니다.

(x, ..., z, i, j) @ (x, ..., z, j, k)을 연산할 때 x, ..., z는 같거나 양쪽 중 한쪽이 1이여야 하며, 맨 끝 두개의 Shape은 (i, j) @ (j, k)에서 j는 동일해야하고, 결과 Shape은 (x, ..., z, i, k)가 된다.
<br>


In [23]:
from numpy.random import rand

print((rand(4, 1, 2) @ rand(4, 2, 5)).shape)
print((rand(1, 1, 2) @ rand(3, 2, 5)).shape)
print((rand(4, 3, 1, 2) @ rand(4, 3, 2, 5)).shape)
print((rand(4, 5, 1, 2) @ rand(4, 1, 2, 5)).shape)
print((rand(4, 5, 1, 2) @ rand(1, 5, 2, 5)).shape)

(4, 1, 5)
(3, 1, 5)
(4, 3, 1, 5)
(4, 5, 1, 5)
(4, 5, 1, 5)


<br>

### 3. Axis

Axis는 축이라는 뜻입니다. Shape에서 몇번째 Shape을 지정할지 나타낼 때 많이 사용합니다. 가령 (4, 1, 3)의 Shape을 가진 텐서가 있다면, Axis 0은 4이며, Axis 1은 1, Axis 2는 3이 됩니다. 몇번째 자리수를 나타내는지 사용할 때 씁니다. <br><br>

### 4. 텐서 및 행렬 연산
텐서나 행렬은 매우 다양한 연산을 지원합니다. 아래의 구체적인 예시를 봅시다.
<br>

#### 4.1. transpose (전치행렬)
$\begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \end{bmatrix}^T$ = $\begin{bmatrix} 1 & 4 \\ 2 & 5 \\ 3 & 6 \end{bmatrix}$

트랜스포즈 연산은 행렬을 뒤집는 것을 의미합니다. 만약 Shape가 (2, 3)인 행렬이였다면 뒤집고나서는 Shape가 (3, 2)가 됩니다. 수식 표기는 $A^T$와 같이 T를 붙여서 표기합니다. numpy에서는 .T로 손쉽게 행렬을 트랜스포즈 할 수 있습니다. 

In [28]:
import numpy as np

mat_a = np.array([[1, 2, 3], 
                  [4, 5, 6]])

mat_a.T

array([[1, 4],
       [2, 5],
       [3, 6]])

<br>

텐서의 경우, 랭크가 2보다 높을 수 있습니다. 각 자리의 순서를 바꾸고 싶을 때 transpose() 연산을 사용할 수 있습니다. 랭크가 3일 때 기본값은 (0, 1, 2)으로 생각하고, 0번째 1번째 2번째 Shape의 위치를 마음대로 변경합니다. 예를 들어 numpy에서는 transpose(0, 2, 1)이나 transpose(1, 0, 2)처럼 변경 가능합니다.
<br><br>

- Shape : (4, 5, 7) ← ([0]:4, [1]:5, [2]:7)
- transpose(0, 1, 2) : (4, 5, 7)
- transpose(0, 2, 1) : (4, 7, 5)
- transpose(1, 2, 0) : (7, 5, 4)
- transpose(1, 0, 2) : (5, 4, 7)

In [42]:
mat_a = np.array([[[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]]])


print('Shape : ', mat_a.shape)
print('transpose(0, 1, 2) : ', mat_a.transpose(0, 1, 2).shape)
print('transpose(0, 2, 1) : ', mat_a.transpose(0, 2, 1).shape)
print('transpose(1, 0, 2) : ', mat_a.transpose(1, 0, 2).shape)
print('transpose(1, 2, 0) : ', mat_a.transpose(1, 2, 0).shape)

Shape :  (4, 2, 3)
transpose(0, 1, 2) :  (4, 2, 3)
transpose(0, 2, 1) :  (4, 3, 2)
transpose(1, 0, 2) :  (2, 4, 3)
transpose(1, 2, 0) :  (2, 3, 4)


<br><br>

#### 4.2. reshpae (shape 및 Rank 변경)
![](https://i.stack.imgur.com/OUjlv.png)
<br>

reshape 연산은 행렬이나 텐서의 일부분을 떼내서 다른 모양으로 재조직합니다. 때문에 Shape을 마음대로 바꿀 수 있습니다. 단 일정한 규칙 안에서 변경 가능한데, 해당하는 텐서의 Shape에서 각 Axis들의 곱으로 만들어낼 수 있는 Shape만 변경 가능합니다. numpy에서는 reshape()이라는 함수로 사용 가능합니다. 아래 예시를 봅시다.
<br><br>

- (4, 2, 3) → reshape(4, 2 x 3) → (4, 6)
- (4, 3, 2) → reshape(4 x 2, 3) → (8, 3)
- (4, 3, 2) → reshape(4 x 2 x 3, 1) → (24, 1)
- (4, 3, 2) → reshape(1, 4 x 2 x 3) → (1, 24)
- (4, 3, 2) → reshape(4 x 2 x 3) → (24)

In [47]:
mat_a = np.array([[[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]]])


print('Shape : ', mat_a.shape)
print('reshape(4, 2 * 3) : ', mat_a.reshape(4, 2 * 3).shape)
print('reshape(4 * 2, 3) : ', mat_a.reshape(4 * 2, 3).shape)
print('reshape(4 * 2 * 3) : ', mat_a.reshape(4 * 2 * 3).shape)
print('reshape(1, 4 * 2 * 3) : ', mat_a.reshape(1, 4 * 2 * 3).shape)
print('reshape(4 * 2 * 3, 1) : ', mat_a.reshape(4 * 2 * 3, 1).shape)

Shape :  (4, 2, 3)
reshape(4, 2 * 3) :  (4, 6)
reshape(4 * 2, 3) :  (8, 3)
reshape(4 * 2 * 3) :  (24,)
reshape(1, 4 * 2 * 3) :  (1, 24)
reshape(4 * 2 * 3, 1) :  (24, 1)


<br>

#### 4.3. squeeze (쥐어 짜기)
squeeze연산은 크기가 1인 Axis를 제거합니다. 텐서를 쥐어짜서 필요 없는 차원을 없앤다고 봐도 무방합니다 (그래서 이름이 squeeze인듯 합니다.)

- Shape : (4, 1, 2) → squeeze : (4, 2)
- Shape : (4, 1, 1, 1, 2) → squeeze : (4, 2)
- Shape : (1, 4, 1, 2) → squeeze : (4, 2)

In [53]:
from numpy.random import rand


print('origin : ', rand(4, 1, 2).shape, 'squeeze : ', rand(4, 1, 2).squeeze().shape)
print('origin : ', rand(4, 1, 1, 1, 2).shape, 'squeeze : ', rand(4, 1, 1, 1, 2).squeeze().shape)
print('origin : ', rand(1, 4, 1, 2).shape, 'squeeze : ', rand(1, 4, 1, 2).squeeze().shape)

origin :  (4, 1, 2) squeeze :  (4, 2)
origin :  (4, 1, 1, 1, 2) squeeze :  (4, 2)
origin :  (1, 4, 1, 2) squeeze :  (4, 2)


<br>

#### 4.4. expand_dims (= unsqueeze, 차원 증가)
expand_dims연산은 unsqueeze라고도 부르는데 squeeze와 반대로 원하는 Axis에 크기가 1인 Axis를 새로 만듭니다. numpy에서는 np.expand_dims(a, 0)과 같이 사용합니다.

- Shape : (4, 2) → expand_dims(0) : (1, 4, 2)
- Shape : (4, 2) → expand_dims(1) : (4, 1, 2)
- Shape : (4, 2) → expand_dims(2) : (4, 2, 1)

In [55]:
from numpy.random import rand


print('expand_dims(0) : ', np.expand_dims(rand(4, 2), axis=0).shape)
print('expand_dims(1) : ', np.expand_dims(rand(4, 2), axis=1).shape)
print('expand_dims(2) : ', np.expand_dims(rand(4, 2), axis=2).shape)

expand_dims(0) :  (1, 4, 2)
expand_dims(1) :  (4, 1, 2)
expand_dims(2) :  (4, 2, 1)


<br>

#### 4.5. concatenate (접합)
![](https://www.w3resource.com/w3r_images/python-numpy-image-exercise-58.png)

두 텐서를 이어 붙여서 새로운 텐서를 만듭니다. 가령 (4, 1, 2)와 (4, 1, 2)를 이어붙여서 (8, 1, 2)와 같이 만듭니다. 어디에 이어 붙일지는 axis로 선택 가능합니다. numpy에서는 np.concatenate([a, b, c, ...], axis=n)과 같이 사용합니다. 전체에서 가장 많이 쓰이는 연산 중 하나이니 꼭 숙지해야합니다.

In [60]:
from numpy.random import rand

tensor_a = rand(4, 1, 2)
tensor_b = rand(4, 1, 2)

print(np.concatenate([tensor_a, tensor_b], axis=0).shape)
print(np.concatenate([tensor_a, tensor_b], axis=1).shape)
print(np.concatenate([tensor_a, tensor_b], axis=2).shape)

(8, 1, 2)
(4, 2, 2)
(4, 1, 4)


<br>

#### 4.6. split (자르기)
![](https://www.w3resource.com/w3r_images/numpy-manipulation-split-function-image-a.png)

concatenate와 반대로 텐서를 여러개로 자릅니다. axis=m에서 n개로 자르려면 numpy에서는 np.split(a, n, axis=m)처럼 사용합니다. concatenate처럼 자주 사용되니 알아둬야합니다.


In [82]:
from numpy.random import rand

tensor_a = rand(4, 1, 2)
print('원본 Shape: ', tensor_a.shape)

splited = np.split(tensor_a, 2) # 2개로 자름
print('axis=0에서 2개로 자름 : ', [s.shape for s in splited])

splited = np.split(tensor_a, 4) # 4개로 자름
print('axis=0에서 4개로 자름 : ', [s.shape for s in splited])

splited = np.split(tensor_a, 2, axis=2) # axis=2에서 2개로 자름
print('axis=2에서 2개로 자름 : ', [s.shape for s in splited])

원본 Shape:  (4, 1, 2)
axis=0에서 2개로 자름 :  [(2, 1, 2), (2, 1, 2)]
axis=0에서 4개로 자름 :  [(1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2)]
axis=2에서 2개로 자름 :  [(4, 1, 1), (4, 1, 1)]


<br><br>

### 5. 텐서 생성 연산
기본적으로 지금까지 numpy.array([1, 2, 3])과 같은 방식으로 numpy 벡터/행렬/텐서 등을 직접 지정해서 생성했습니다. 그러나 이 방법 말고도 다양한 생성 방법이 존재합니다.
<br>

#### 5.1. rand
위 예시에서 자주 사용했던 rand입니다. 주어진 Shape로 랜덤 값을 생성합니다.
np.random.rand(shape)와 같이 사용할 수 있습니다.

In [96]:
import numpy as np

np.random.rand(2, 2, 3)

array([[[0.40423873, 0.20384893, 0.40205417],
        [0.19334684, 0.06202334, 0.94376573]],

       [[0.23077679, 0.15761586, 0.84891135],
        [0.39147286, 0.18499911, 0.2066128 ]]])

<br>

#### 5.2. ones
주어진 Shape로 1로 초기화된 텐서를 생성합니다. np.ones((shape))와 같이 사용합니다. 주의할 점은 ()괄호로 shape을 한번 감싸줘야합니다.

In [87]:
np.ones((2, 2, 3))

array([[[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]]])

<br>

#### 5.2. zeros
주어진 Shape로 0로 초기화된 텐서를 생성합니다. np.zeros((shape))와 같이 사용합니다. 주의할 점은 ()괄호로 shape을 한번 감싸줘야합니다.

In [94]:
np.zeros((2, 2, 3))

array([[[0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.]]])

<br>

#### 5.3. arrange
주어진 크기로 점점 증가하는 행 벡터를 생성합니다. np.arrange(size)와 같이 사용합니다.

In [93]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

<br>


### 6. Distance & Norm
Distance는 텐서 사이의 거리, Norm은 텐서의 길이를 나타냅니다. 마지막으로 이들에 대해 학습해봅시다.
<br><br>

#### 6.1. Distance
Distance는 말 그대로 텐서와 텐서 사이의 거리를 나타냅니다. 일반적으로 두 점 사이의 거리(유클리디안 거리)는 아래와 같이 나타낼 수 있습니다.

![](https://mblogthumb-phinf.pstatic.net/MjAxODA0MjhfMTUy/MDAxNTI0OTE2NzE1MDE2.AM35pyBBlAll3OsKna2d4bX4GMoifemnj0Wy5VRrKIwg.dYmu6wzSk3RqjTCilmVVnizYbPENZkZxZnORZj0ZybYg.JPEG.galaxyenergy/1524914716171.jpg?type=w800)
<br><br>

예를 들어 $\begin{bmatrix} 2 \\ 3 \end{bmatrix}$와  $\begin{bmatrix} 5 \\ 10 \end{bmatrix}$ 같은 벡터가 있다면 두 벡터 사이의 거리는 $\sqrt{(5 - 2)^2 + (10 - 3)^2}$이 됩니다. 이런 거리를 L2거리, 혹은 유클리디안 거리라고 합니다. 그러면 유클리디안 거리가 아닌 거리도 있을까요? 제곱을 하지 않고 절대값을 사용하는 L1거리, 맨하탄 거리도 있습니다. 아래를 볼까요
<br><br>

![](https://lh3.googleusercontent.com/proxy/AVIWU_vSjlU9Dd8wcYOAAdD7ZkrsQeGjTitT0MSDhTPHqqSkb4EkqGuJ1lrSH_VTD0gr66URFHxYPxLAf-4dJnn6umoY7Fikdn0)
<br><br>

맨하탄 거리는 약간 다릅니다. $\begin{bmatrix} 2 \\ 3 \end{bmatrix}$와  $\begin{bmatrix} 5 \\ 10 \end{bmatrix}$ 같은 벡터가 있다면 두 벡터 사이의 맨하탄 거리는 $|5 - 2| + |10 - 3|$이 됩니다. 이렇게 구하는 거리를 맨하탄 거리, 혹은 L1 거리라고 합니다. 
<br><br>

![](https://mblogthumb-phinf.pstatic.net/MjAxNzA0MTJfMjQ2/MDAxNDkxOTY0NDkyNzUw.pcEqq7JbtHYh31aVcK3RF7xvlOWCfbnV77A9GDyKyZ8g.9L1Un6iGL7n0Slpnrdp57GQS0zecsXTM6jypv72Vk4kg.PNG.samsjang/%EC%BA%A1%EC%B2%98.PNG?type=w2)

길어지는 식을 시그마를 이용해 간단하게 나타낼 수 있습니다. 맨하탄 거리는 $\sum |a_1 - a_2|$를, 유클리디언 거리는 $\sum \sqrt{(a_1 - a_2)^2}$로 나타낼 수 있습니다. 그렇다면 $\sum \sqrt[3]{(a_1 - a_2)^3}$이나 $\sum \sqrt[4]{(a_1 - a_2)^4}$같은 거리는 없을까요? 있습니다. 이런 거리들을 전부 일반화 시킨 것을 Minkowski(민코프스키) 거리라고 합니다. 민코프스키 거리는 $\sum \sqrt[p]{(a_1 - a_2)^p}$와 같이 나타내고 P = 1일때는 맨하탄 거리, P = 2일때는 유클리디언 거리가 됩니다.
<br><br>


#### 6.2. Norm
Norm(노름)은 벡터나 텐서등의 길이를 나타낼 때 씁니다. Norm은 위의 거리 공식 $distance = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$ 에서 유도할 수 있습니다. 그러나 앞서 배웠듯이 벡터는 원점에서 한 점으로 뻗어나온 직선입니다. 때문에 둘 중 한점의 좌표는 원점(0, 0)으로 생각할 수 있고 이는 즉 $norm = \sqrt{x_2^2 + y_2^2}$와 같이 식을 변경할 수 있습니다. 
<br><br>

무슨 말이냐면, 예를 들어 $\begin{bmatrix} 2 \\ 3 \end{bmatrix}$과 같은 벡터가 있다면 원점과의 거리(길이)는 $\sqrt{(0 - 2)^2 + (0-3)^2}$인데, (0, 0)은 계산하는 의미가 없으므로 그냥 $\sqrt{2^2 + 3^2}$가 된다는 것입니다. Norm도 역시 마찬가지로 L1 Norm, L2 Norm 등이 존재하는데, 원점과의 거리를 계산할 때 유클리디언 거리로 계산하면 L2 Norm이 되고, 맨하탄 거리로 계산하면 L1 Norm이 됩니다.
<br><br>

norm은 numpy의 linalg 패키지에 있는 norm 함수로 매우 쉽게 계산할 수 있습니다. 이 때, ord는 minkowski의 p를 몇으로 설정하지에 관한 것으로 ord가 2이면 유클리디언 거리 ord가 1이면 맨하탄 거리가 됩니다.

In [109]:
import numpy as np


vec_a = np.array([2, 3])

# linalg는 Linear Algebra의 줄임말입니다.
np_l2_norm = np.linalg.norm(vec_a, ord=2)
sc_l2_norm = np.sqrt((vec_a[0] - 0) ** 2 + (vec_a[1] - 0) ** 2)

# L2 Norm (유클리디안 길이) 계산
print('numpy_l2_norm : ', np_l2_norm)
print('scratch_l2_norm : ', sc_l2_norm)

numpy_l2_norm :  3.605551275463989
scratch_l2_norm :  3.605551275463989


In [111]:
import numpy as np


vec_a = np.array([2, 3])

# linalg는 Linear Algebra의 줄임말입니다.
np_l1_norm = np.linalg.norm(vec_a, ord=1)
sc_l1_norm = (vec_a[0] - 0) + (vec_a[1] - 0)

# L1 Norm (맨하탄 길이) 계산
print('numpy_l1_norm : ', np_l1_norm)
print('scratch_l1_norm : ', sc_l1_norm)

numpy_l1_norm :  5.0
scratch_l1_norm :  5


<br>

두 벡터 사이의 Distance를 구하려면 